In [26]:
from random import random, randint
import datetime

from meilisearch import Client
from pymongo import MongoClient

In [27]:
client = Client("http://127.0.0.1:7700", "masterKey")

In [28]:
tanks = client.index("tanks")

In [30]:
before = tanks.get_documents({"limit": 2000})

sizes = ["1.5L", "3L", "10L"]
sexual_list = ["male", "female"]
genotypes = [
    "genotype-a",
    "genotype-b",
    "genotype-c",
    "genotype-d",
    "genotype-e",
    "genotype-f",
    "genotype-g"
]
species = ["species-a", "species-b", "species-c", "species-d", "species-e"]

for i in before:
    i["size"] = sizes[randint(0, 2)]
    if random() < 0.4:
        i["sexual"] = sexual_list[randint(0, 1)]
    if random() < 0.3:
        i["genotype"] = genotypes[randint(0, 6)]
    if random() < 0.35:
        i["species"] = species[randint(0, 4)]

tanks.update_documents(before)

In [32]:
all_tanks = tanks.get_documents({"limit": 2000})

In [33]:
records = MongoClient("mongodb://localhost:27017").get_database("killifish").get_collection("records")

In [21]:
data = []

In [34]:
begin = datetime.datetime(2022, 1, 1, 6, 0, 0)
end = datetime.datetime(2022, 12, 30, 6, 0, 0)

d = begin
delta = datetime.timedelta(hours=8)

while d < end:

    for tank in all_tanks:
        tmp = {
            "time": d.strftime("%Y-%m-%d %H:%M:%S"),
            "owner": tank["owner"],
            "tankId": tank["id"],
            "quantity": randint(1, 3) * tank["amount"],
            "genotype": tank.get("genotype"),
            "sexual": tank.get("sexual"),
            "birthday": tank.get("birthday")
        }

        data.append(tmp)

    d += delta

In [35]:
len(data)

1078110

In [36]:
records.insert_many(data)